In [1]:
import itertools

In [20]:
list(list(itertools.product([1,2], repeat=2)))

[(1, 1), (1, 2), (2, 1), (2, 2)]

___
Давайте вернёмся к данным выживаемости пациентов с лейкоцитарной лимфомой из видео про критерий знаков:
49,58,75,110,112,132,151,276,281,362 

Измерено остаточное время жизни с момента начала наблюдения (в неделях); звёздочка обозначает цензурирование сверху — исследование длилось 7 лет, и остаточное время жизни одного пациента, который дожил до конца наблюдения, неизвестно.

Поскольку цензурировано только одно наблюдение, для проверки гипотезы 
H0:medX=200 на этих данных можно использовать критерий знаковых рангов — можно считать, что время дожития последнего пациента в точности равно 362, на ранг этого наблюдения это никак не повлияет.

Критерием знаковых рангов проверьте эту гипотезу против двусторонней альтернативы, введите достигаемый уровень значимости, округлённый до четырёх знаков после десятичной точки.

In [1]:
import scipy.stats as sts
import numpy as np

In [30]:
sample = np.array([49,58,75,110,112,132,151,276,281,362])
m0 = 200

In [31]:
sts.wilcoxon(sample - m0)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

___
В ходе исследования влияния лесозаготовки на биоразнообразие лесов острова Борнео собраны данные о количестве видов деревьев в 12 лесах, где вырубка не ведётся:

22,22,15,13,19,19,18,20,21,13,13,15

и в 9 лесах, где идёт вырубка:


17,18,18,15,12,4,14,15,10.

Проверьте гипотезу о равенстве среднего количества видов в двух типах лесов против односторонней альтернативы о снижении биоразнообразия в вырубаемых лесах. Используйте ранговый критерий. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

In [34]:
no_deforest = np.array([22,22,15,13,19,19,18,20,21,13,13,15])
deforest = np.array([17,18,18,15,12,4,14,15,10])
sts.mannwhitneyu(no_deforest, deforest, alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.02900499272087373)

___
28 января 1986 года космический шаттл "Челленджер" взорвался при взлёте. Семь астронавтов, находившихся на борту, погибли. В ходе расследования причин катастрофы основной версией была неполадка с резиновыми уплотнительными кольцами в соединении с ракетными ускорителями. Для 23 предшествовавших катастрофе полётов "Челленджера" известны температура воздуха и появление повреждений хотя бы у одного из уплотнительных колец.

С помощью бутстрепа постройте 95% доверительный интервал для разности средних температур воздуха при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после запятой.

Чтобы получить в точности такой же доверительный интервал, как у нас:

установите random seed = 0 перед первым вызовом функции get_bootstrap_samples, один раз
сделайте по 1000 псевдовыборок из каждой выборки.

In [71]:
import pandas as pd
df = pd.read_csv('data/challenger.txt', sep='\t')
df.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [159]:
df[df['Incident'] == 0].describe()

,Temperature,Incident
count,16.000000,16.0
mean,22.281250,0.0
std,2.693503,0.0
min,18.900000,0.0
25%,19.850000,0.0
50%,21.650000,0.0
75%,24.400000,0.0
max,27.200000,0.0


In [160]:
df[df['Incident'] == 1].describe()

,Temperature,Incident
count,7.000000,7.0
mean,17.614286,1.0
std,4.529322,0.0
min,11.700000,1.0
25%,14.150000,1.0
50%,17.200000,1.0
75%,21.100000,1.0
max,23.900000,1.0


In [72]:
def get_bootstrap_samples(data, n_samples):
    inds = np.random.randint(0, len(data), (n_samples, len(data)))
    return data[inds]

In [161]:
noinc_temp = df[df['Incident'] == 0].Temperature.values
inc_temp = df[df['Incident'] == 1].Temperature.values

In [171]:
np.random.seed(0)
noinc_temp_brp = get_bootstrap_samples(noinc_temp, 1000)
inc_temp_brp = get_bootstrap_samples(inc_temp, 1000)

In [172]:
temp_diffs = inc_temp_brp.mean(axis=1) - noinc_temp_brp.mean(axis=1)

In [173]:
import statsmodels.stats.weightstats as wsts

In [179]:
alpha = 0.05
np.percentile(-temp_diffs, [100 * alpha / 2., 100 * (1 - alpha / 2.)])

array([1.42299107, 7.93861607])

___
На данных предыдущей задачи проверьте гипотезу об одинаковой средней температуре воздуха в дни, когда уплотнительный кольца повреждались, и дни, когда повреждений не было. Используйте перестановочный критерий и двустороннюю альтернативу. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

Чтобы получить такое же значение, как мы:

установите random seed = 0;
возьмите 10000 перестановок.

In [147]:
import permutation_criteria

In [214]:
np.random.seed(0)
permutation_criteria.permutation_test_2sample_ind(
    inc_temp, noinc_temp, max_permutations=10000)

0.0057